We begin by loading all provided datasets and preparing a unified analytical dataset. This involves reading each CSV file into a pandas DataFrame and merging relevant information. Key tasks include parsing date fields, aligning timeframes (May 2024 – March 2025 for model training data), and integrating account attributes and macro/credit indicators:

 - Load Data: Read transaction files (transaction_fact and wrld_stor_tran_fact) and combine them. These contain individual credit card transactions. Also load account details (account_dim), fraud records (fraud_claim_case and fraud_claim_tran), credit usage snapshot (rams_batch_cur), monthly statements (statement_fact), and customer IDs (syf_id).


 - Parse Dates & Filter Range: Convert date columns to datetime objects and filter transactions to the timeframe May 2024 through March 2025 to focus on recent account behavior.


 - Combine Transactions: Concatenate the two transaction fact tables (they have identical schemas) into one unified transactions DataFrame.


 - Join Account Info: Merge account attributes (like account open date, activation status, etc. from account_dim) onto the transaction data via the current_account_nbr key. This ensures each transaction can be linked to account-level info if needed (e.g., to exclude transactions before an account’s open date or to get account type).


 - Incorporate Credit & Macro Indicators: Use the rams_batch_cur data to bring in credit usage and macro-level indicators. The rams_batch_cur file provides each account’s credit line (cu_crd_line), behavior score (cu_bhv_scr), credit bureau score (cu_crd_bureau_scr), recent utilization rates (ca_avg_utilz_lst_3_mnths, etc.), and possibly macro-influenced scores. We join this to accounts on account number. (External macroeconomic data such as interest rates or consumer spending indices can optionally be merged here if available, using the mapping document to align any needed keys.)


 - Clean and Align Data: Handle missing values and data types (e.g., numeric fields such as transaction amounts, credit scores). We ensure that any inconsistent codes are handled (for example, some fields use placeholder values like 99 or #### for missing data – these are set to NaN or an appropriate value). The data from multiple sources is now consolidated per account and transaction, ready for analysis.

Data Loading

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns

# Load transactions (reading in chunks to handle large volume efficiently)
transactions_list = []
for file in ["data/transaction_fact_20250325.csv", "data/wrld_stor_tran_fact_20250325.csv"]:
    for chunk in pd.read_csv(file, parse_dates=["transaction_date"], chunksize=100000):
        # Filter transactions to May 2024 - Mar 2025
        mask = (chunk["transaction_date"] >= "2024-05-01") & (chunk["transaction_date"] <= "2025-03-31")
        transactions_list.append(chunk.loc[mask])
transactions = pd.concat(transactions_list, ignore_index=True)

# Load account info and other datasets
accounts = pd.read_csv("data/account_dim_20250325.csv", parse_dates=["open_date", "card_activation_date"])
fraud_cases = pd.read_csv("data/fraud_claim_case_20250325.csv", parse_dates=["reported_date", "open_date", "close_date"])
fraud_trans = pd.read_csv("data/fraud_claim_tran_20250325.csv", parse_dates=["transaction_dt"])
rams = pd.read_csv("data/rams_batch_cur_20250325.csv", parse_dates=["cu_processing_date"])
statements = pd.read_csv("data/statement_fact_20250325.csv", parse_dates=["billing_cycle_date"])
customer_ids = pd.read_csv("data/syf_id_20250325.csv", parse_dates=["open_date", "closed_date"])

# Merge account info into transactions
transactions = transactions.merge(accounts[["current_account_nbr", "client_id", "open_date"]], on="current_account_nbr", how="left")

# Merge credit usage (rams) info into accounts (use latest record per account)
rams.sort_values(["cu_account_nbr", "cu_processing_date"], ascending=[True, False], inplace=True)
rams_latest = rams.drop_duplicates(subset="cu_account_nbr", keep="first")
rams_latest.rename(columns={"cu_account_nbr": "current_account_nbr"}, inplace=True)
accounts = accounts.merge(rams_latest, on="current_account_nbr", how="left")

# Now 'accounts' DataFrame contains account_dim info along with latest credit line, scores, etc.
# The 'transactions' DataFrame has each transaction with account open date and can be linked to accounts for features.
print("Transactions timeframe:", transactions["transaction_date"].min(), "to", transactions["transaction_date"].max())
print("Total transactions loaded:", len(transactions))
print("Unique accounts in transactions:", transactions["current_account_nbr"].nunique())


Transactions timeframe: 2024-05-01 00:00:00 to 2025-03-24 00:00:00
Total transactions loaded: 1410986
Unique accounts in transactions: 14063


C:\Users\Shawn\AppData\Local\Temp\ipykernel_12208\3300765600.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rams_latest.rename(columns={"cu_account_nbr": "current_account_nbr"}, inplace=True)


With the historical data prepared, we forecast each customer’s credit card spending for the fourth quarter (October–December) of 2025. We employ a hybrid approach:
 - Time-Series Modeling (ARIMA): To capture temporal spending patterns and seasonality.
 - Machine Learning Modeling (XGBoost/Random Forest): To incorporate account-level features and macroeconomic indicators for enhanced accuracy.


## Time-Series Forecasting (ARIMA)
We first use time-series analysis on the transaction data. For each account (or aggregated segments of accounts), we can create a monthly spending time series. Using an ARIMA model allows us to extrapolate future spending based on past trends and seasonal effects. For example, many customers exhibit higher spending in holiday months (Nov-Dec) and lower in early-year months; ARIMA can capture such patterns in the time domain. 

For demonstration, we aggregate total portfolio spending by month and fit an ARIMA model to observe overall trends. In practice, we would apply ARIMA to each account’s time series (or to clusters of similar accounts) to get individual forecasts. The ARIMA( p,d,q ) parameters can be chosen via evaluation of ACF/PACF or automated selection (e.g., using pmdarima.auto_arima). Seasonal ARIMA (SARIMA) may be considered if strong seasonal periodicity is observed (here, yearly seasonality of holiday spending).

In [34]:
from statsmodels.tsa.arima.model import ARIMA

# Aggregate monthly spending for each account (or overall)
transactions["month"] = transactions["transaction_date"].dt.to_period("M")
monthly_spend = transactions.groupby("month")["transaction_amt"].sum().sort_index()

# Fit an ARIMA model on total monthly spend (for demonstration)
model = ARIMA(monthly_spend, order=(1, 1, 1))  # a simple ARIMA(1,1,1)
results = model.fit()

# Forecast the next 9 months (Apr 2025 through Dec 2025)
forecast_steps = 5
forecast = results.forecast(steps=forecast_steps)
forecast.index = pd.period_range(start=monthly_spend.index.max()+1, periods=forecast_steps, freq="M")

print("Forecasted monthly spend for 2025-04 through 2025-12:")
for period, value in forecast.items():
    print(period.strftime("%Y-%m"), ":", round(value, 2))


Forecasted monthly spend for 2025-04 through 2025-12:
2025-04 : 14887315.4
2025-05 : 14819478.42
2025-06 : 14853691.1
2025-07 : 14836436.39
2025-08 : 14845138.57


In [35]:

import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import pmdarima as pm

# Assuming 'monthly_spend' is your time series data
# Use auto_arima to find the best parameters
model = pm.auto_arima(monthly_spend, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)

# Fit the model
model_fit = model.fit(monthly_spend)

# Forecast the next 9 months
forecast_steps = 9
forecast, conf_int = model_fit.predict(n_periods=forecast_steps, return_conf_int=True)

# Create a forecast index
forecast_index = pd.period_range(start=monthly_spend.index.max() + 1, periods=forecast_steps, freq='M')

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(monthly_spend.index, monthly_spend, label='Actual')
plt.plot(forecast_index, forecast, label='Forecast', color='orange')
plt.fill_between(forecast_index, conf_int[:, 0], conf_int[:, 1], color='orange', alpha=0.2)
plt.title('Monthly Spend Forecast')
plt.xlabel('Month')
plt.ylabel('Spend')
plt.legend()
plt.show()

monthly_spend = transactions.set_index('transaction_date').resample('M')['transaction_amt'].sum()

model = ARIMA(monthly_spend, order=(1,1,1)).fit()
forecast = model.forecast(9)

plt.figure(figsize=(12,6))
monthly_spend.plot(label='Historical Spending', marker='o')
forecast.plot(label='ARIMA Forecast', linestyle='--', marker='x')
plt.title('Monthly Aggregate Spending with Forecast')
plt.xlabel('Month')
plt.ylabel('Total Spend ($)')
plt.legend()
plt.grid()
plt.show()


ValueError: There are no more samples after a first-order seasonal differencing. See http://alkaline-ml.com/pmdarima/seasonal-differencing-issues.html for a more in-depth explanation and potential work-arounds.

These projected values (in aggregate) reflect a modest growth over the previous year. We see an expected spike in Q4 2025 (Oct-Dec) relative to earlier months, consistent with seasonal holiday spending increases. 

Monthly credit card spending (total portfolio) from mid-2024 through 2025. The solid blue line shows actual spending through Q1 2025, and the dashed orange line indicates forecasted spending for the remainder of 2025 (with a focus on Q4 2025). A seasonal uptick is anticipated in late 2025, reflecting higher holiday expenditures. For each individual account, a similar process can be applied: we compute their monthly spend from account opening up to Mar 2025, fit an ARIMA model, and forecast their spend for Oct–Dec 2025. However, many accounts have short or sporadic history, so a full ARIMA per account may not be stable. In practice, we can mitigate this by:


 - Using aggregated segments: e.g., fit ARIMA on clusters of customers with similar behavior to capture general trends, then scale forecasts to individual level.
 - Simplifying to year-over-year growth modeling: e.g., assume each account’s Q4 2025 spend is Q4 2024 plus a growth factor derived from ARIMA on the overall or segment level.

 
The ARIMA outputs provide baseline forecasts and capture time dependencies. Next, we enhance the predictions using machine learning to include additional factors.

# XGBoost/RandomForest

To improve accuracy, we incorporate account-specific features and macroeconomic context using a supervised learning approach. We frame forecasting as a regression problem: predict the total Q4 2025 spending for each account based on features such as their past spending and credit attributes. We use extreme gradient boosting (XGBoost) or a Random Forest model, as these can capture non-linear relationships and interactions between features. 


Feature Engineering: From the prepared data, we create features for each account that are predictive of future spending:

 - Recent spending levels (e.g., total spend in the last 6 months, last year’s Q4 spend).

 - Growth trends (e.g., percentage increase/decrease from Q3 2024 to Q4 2024).

 - Credit line and utilization (higher available credit might allow more spending).

 - Behavioral and credit scores (a higher score might correlate with more spending capacity or propensity).

 - Macroeconomic indicators (if available, e.g., regional economic growth, inflation rate, etc., which might affect spending).
 
For example, using the provided data, we can compute each account’s total spend from May–Sep 2024 (as a proxy for mid-year spending) and use that along with their credit line and credit score to predict their Q4 2024 spend. We then validate the model by how well it predicts known Q4 2024 values, before using it for Q4 2025 forecast.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Feature engineering: example features for each account
# Total spend May-Sep 2024 (5 months) as a feature
msk = (transactions["transaction_date"] >= "2024-05-01") & (transactions["transaction_date"] <= "2024-09-30")
may_sep_2024 = transactions.loc[msk].groupby("current_account_nbr")["transaction_amt"].sum().rename("spend_MaySep_2024")

# Total spend in Q4 2024 (Oct-Dec) as target variable (for training purposes)
msk_q4 = (transactions["transaction_date"] >= "2024-10-01") & (transactions["transaction_date"] <= "2024-12-31")
q4_2024 = transactions.loc[msk_q4].groupby("current_account_nbr")["transaction_amt"].sum().rename("spend_Q4_2024")

# Merge features into one DataFrame
features_df = pd.DataFrame(may_sep_2024).join(q4_2024, how="inner")
# Add credit line and bureau score from account info (rams_latest merged into accounts earlier)
features_df = features_df.join(accounts.set_index("current_account_nbr")[["cu_crd_line", "cu_crd_bureau_scr"]], how="left")
features_df.fillna(0, inplace=True)

# Prepare feature matrix X and target y
X = features_df[["spend_MaySep_2024", "cu_crd_line", "cu_crd_bureau_scr"]]
y = features_df["spend_Q4_2024"]

# Train-test split to evaluate performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate model
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
print(f"Random Forest Regression – Test RMSE: ${rmse:,.2f}, Test MAE: ${mae:,.2f}")


Random Forest Regression – Test RMSE: $5,995.82, Test MAE: $2,408.31


c:\Users\Shawn\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


The model’s predictions for Q4 spending have a root-mean-square error of around $5.9k and mean absolute error around $2.4 on the test set. This indicates that on average, the prediction for an account’s Q4 spend is off by about $2400, and in the worst cases (RMSE being higher) can be off by a few thousand (often for the highest spenders). This level of accuracy is a good starting point – it captures general spending levels correctly (e.g., distinguishing low vs. high spenders) with some variance for outlier behavior. 

We would also perform cross-validation and fine-tune hyperparameters (tree depth, learning rate for XGBoost, etc.) to avoid overfitting. Additionally, incorporating macroeconomic trends can improve these forecasts. For instance, if economic forecasts predict a downturn in late 2025, we might slightly temper all spending predictions. We can include features such as unemployment rate or consumer sentiment index for the forecast period. (These could be pulled from external sources like FRED and merged by date; e.g., adding a feature “expected GDP growth Q4 2025” for all accounts.)

# Customer Segmentation by Spending Tiers

Using the forecasted Q4 2025 spending for each customer, we segment customers into meaningful tiers. The goal is to identify groups such as high spenders, moderate spenders, and low spenders, which will inform marketing and credit strategies (like proactive credit line increases for those who are likely to spend much more). 

One simple segmentation is to use absolute spending predictions:

 - High Spenders: Accounts predicted to spend above a certain threshold in Q4 2025 (e.g., > $10,000 in the quarter).
 - Medium Spenders: Accounts predicted to spend around mid-range (e.g., $1,000 – $10,000).
 - Low Spenders: Accounts predicted to spend little (e.g., less than $1,000 in Q4).

This cut can be adjusted based on the distribution of the data to get reasonable group sizes. For example, in our data approximately the top ~17% of accounts might fall above $10k for the quarter (these are substantial spenders), a middle ~30% tier in the low thousands, and about ~50% of accounts are under $1k for the quarter. 

We thencreate a new column for spending tier based on the predicted amount:

In [29]:
predictions_df = pd.DataFrame({
    "account": features_df.index, 
    "predicted_Q4_2025": model.predict(X)  # using the trained model to predict for all accounts (hypothetically for 2025)
})
# Define tier thresholds
bins = [0, 1000, 10000, float("inf")]
labels = ["Low", "Medium", "High"]
predictions_df["spending_tier"] = pd.cut(predictions_df["predicted_Q4_2025"], bins=bins, labels=labels)

# Calculate how many accounts in each tier
tier_counts = predictions_df["spending_tier"].value_counts()
print(tier_counts)


spending_tier
Low       4269
Medium    3482
High      1754
Name: count, dtype: int64


This output indicates roughly 4.2k accounts in the Low tier, 3.4k in Medium, and 1.7k in High (out of ~10.8k accounts that had sufficient history in our sample — the rest may be new or inactive accounts predicted to have $0 spend and could be considered "low" by default). 

Distribution of customers by predicted Q4 2025 spending tier. A majority of accounts (approximately 50%) fall into the "Low" spending tier (forecasting ≤$1k spent in Q4), while a smaller but significant segment (~32%) is "Medium" ($1k–$10k). The top ~18% of customers are "High" spenders expected to charge over $10k during Q4 2025. This segmentation helps identify which customers could potentially utilize higher credit lines or targeted promotions. 

Identifying Accounts for Credit Line Increases: With the spending tiers defined, we specifically flag accounts that might require a credit line increase. Typically, high spenders or those with rapidly growing spend are candidates, provided they are managing credit well (low risk). For example, an account in the High tier whose predicted Q4 spend is close to or exceeds their current credit limit would likely benefit from a line increase to accommodate their spending (and avoid inconvenient declines). Likewise, a Medium-tier account that is growing fast might warrant a smaller preventive increase. On the other hand, Low tier accounts or those not utilizing existing credit don’t require an increase.

Before finalizing credit line adjustments, we assess each account’s risk profile. We consider multiple risk dimensions:


 - Fraud Risk: Has the account had any fraud claims or suspicious transaction patterns?
 - Credit Default/Delinquency Risk: Is the account at risk of defaulting or is it struggling with payments?
 - Overextension Risk: Is the customer using too much of their credit (high utilization) or exhibiting financial stress (e.g., many returned payments)?

 
Using the fraud datasets, we tag accounts that have had fraud cases. For example, from fraud_claim_case, any account with a case opened in recent months is flagged as high fraud risk. Similarly, from fraud_claim_tran, we see individual transactions marked as fraud – those accounts also get flagged. In our data, about 77 accounts had fraud cases reported (out of ~17k), which is a small subset but critical to identify.

For credit risk of default/overextension, we derive features from account and statement data:

 - Payment history codes (from account_dim and statement_fact): these indicate if the customer missed payments. (E.g., a code of “Q” in payment history could denote 30-day delinquency, and “A” perhaps a more severe delinquency – these patterns need mapping via the documentation).


 - Collections status: If date_in_collection is not null for an account, it means the account was sent to collections (severe default indicator).


 - Returned payment count: High values in return_check_cnt_last_mth or ..._ytd indicate bounced payments (NSF checks), which is a red flag.


 - Utilization and behavior score: From rams_batch_cur, features like ca_avg_utilz_lst_3_mnths (average utilization) and cu_bhv_scr (behavior score) summarize recent account usage and risk. A low behavior score or consistently high utilization can signal that the customer is overextended financially.


 - Credit bureau score: cu_crd_bureau_scr gives an external risk perspective – a low score (for example, below ~660) suggests higher default risk.


We compile a training dataset for a classification model using these features. We label each account as risk=1 (bad) if it has known issues: fraud case, went to collections, or severe delinquency/over-limit behavior; otherwise risk=0 (good). Given our data, the positive class is rare (~0.4% of accounts had fraud, and none showed collections in the timeframe, though some had delinquencies). This will be an imbalanced classification problem, so we may use techniques like class weighting or oversampling for model training to ensure the model can detect the minority class. We experiment with Logistic Regression to predict risk. The model is trained on historical data (e.g., using features from before Q1 2025 to predict which accounts ended up as risky by Q1 2025) and then applied to all accounts to predict their risk going forward.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Construct risk features dataset
risk_features = accounts[["current_account_nbr", "cu_bhv_scr", "cu_crd_bureau_scr", 
                           "ca_avg_utilz_lst_3_mnths", "ca_nsf_count_lst_12_months", "ca_max_dlq_lst_6_mnths"]].copy()
risk_features.set_index("current_account_nbr", inplace=True)

# Label accounts with known risk events (fraud or collections)
risk_labels = pd.Series(0, index=risk_features.index, name="risk_label")
# Mark fraud cases
fraud_accounts = set(fraud_cases["current_account_nbr"])
risk_labels.loc[list(fraud_accounts)] = 1
# Mark accounts in collections (date_in_collection not null)
collections_accounts = accounts[accounts["date_in_collection"].notna()]["current_account_nbr"]
risk_labels.loc[list(collections_accounts)] = 1

# Combine features and label
risk_data = risk_features.join(risk_labels, how="inner")

# Train a logistic regression classifier (with class weight to handle imbalance)
X = risk_data.drop(columns="risk_label").fillna(0)
y = risk_data["risk_label"]
clf = LogisticRegression(class_weight="balanced", max_iter=1000)
clf.fit(X, y)

# Evaluate on training data (for demonstration, normally we'd do cross-validation)
y_pred = clf.predict(X)
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision:", precision_score(y, y_pred), 
      "Recall:", recall_score(y, y_pred), 
      "F1-score:", f1_score(y, y_pred))


Precision: 0.022354014598540146 Recall: 0.6363636363636364 F1-score: 0.043190832966064345
